Name: run_wc_dataset;

Version: 2.0;

Author: Anish Vasan;

Organization: Chen/Eyckmans Lab, Boston University;

Date: 20230509;

## IMPORTS ##

In [33]:
import os, shutil, fnmatch, tkinter, yaml
from tkinter.filedialog import askdirectory
from pathlib import Path
from woundcompute import image_analysis as ia
import time

## DEFINITIONS ##
## These require user input/changes based on experiments ##

In [34]:
image_type = "ph1"  #Options:  brightfield, ph1
is_fl = False #Options: True, False
is_track = False #Options: True, False

## FUNCTIONS ##

In [35]:
def create_yaml(path: str, image_type_in: str):
    """Given the output path as string. Will create a yaml file in the main output folder. This yaml file will be copied into each subfolder during the sorting function"""

    #Default keys and values stored in yaml file
    yaml_input_file = {
    'version' : 1.0,
    'segment_brightfield' : False,
    'seg_bf_version' : 1,
    'seg_bf_visualize' : False,
    'segment_fluorescent' : False,
    'seg_fl_version' : 1,
    'seg_fl_visualize' : False,
    'segment_ph1' : False, #True,
    'seg_ph1_version' : 2,
    'seg_ph1_visualize' : False, #True,
    'track_brightfield' : False,
    'track_bf_version' : 1,
    'track_bf_visualize' : False,
    'track_ph1' : False,
    'track_ph1_version' : 1,
    'track_ph1_visualize' : False,
    'bf_seg_with_fl_seg_visualize' : False,
    'bf_track_with_fl_seg_visualize' : False,
    'ph1_seg_with_fl_seg_visualize' : False,
    'ph1_track_with_fl_seg_visualize' : False,
    }

    #Conditionally modify yaml file based on image_type input

    for key, value in yaml_input_file.items():
        if image_type_in in key and not "version" in key and not "fl" in key and not "track" in key:
            yaml_input_file[key] = True

            if is_fl:
                if "fl" in key and not "version" in key:
                    yaml_input_file[key] = True

            if is_track:
                if "track" in key and not "version" in key:
                    yaml_input_file[key] = True

        if image_type_in == "brightfield" :
            yaml_input_file['seg_bf_visualize'] = True

    # Write yaml output to path_output
    with open(path+'/wc_dataset_'+image_type_in+'.yaml', 'w') as file:
        yaml.safe_dump(yaml_input_file, file, sort_keys=False)



In [36]:
def sort_stage_pos_folders(input_fn: str,output_fn: str, image_type_fn: str):
    """Given basename list, parent output folder and number of stage positions list extracted from nd files. Sorts files into stage position folders"""
    #sort images into stage position folders
    for file in os.listdir(input_fn):

        if file.endswith(".tif"):
            #Check if position described with only 2 characters
            if file[2] == '_':
                file_sortname = file[0] + f"{int(file[1]):02}"


            else:
                file_sortname = file[:3]

            file_newname = file_sortname +"_"+ file[-7:]


            if os.path.exists(output_fn + '/' + file_sortname + '/' +  image_type_fn+ '_images/'):
                try:
                    shutil.copy(input_fn + '/' + file, output_fn + '/' + file_sortname + '/' +image_type_fn+ '_images/'+ file_newname)
                except OSError as e:
                    # If it fails, inform the user.
                    print('Error: %s - %s.' % (e.filename, e.strerror))

            else:
                try:
                    os.makedirs(output_fn + '/' + file_sortname + '/' +  image_type_fn+ '_images/')
                    shutil.copy(output_fn + '/wc_dataset_'+image_type_fn+'.yaml', output_fn + '/' + file_sortname + '/wc_dataset_'+image_type_fn+'.yaml')
                    shutil.copy(input_fn + '/' + file, output_fn + '/' + file_sortname +'/' + image_type_fn+ '_images/'+ file_newname)


                except OSError as e:
                    # If it fails, inform the user.
                    print('Error: %s - %s.' % (e.filename, e.strerror))


## RUN CODE ##

**Section 1: Navigate to data folder containing .nd file**

In [37]:
print("Current config:")
print("Image Type:", image_type)
print("Fluorescent Images:", is_fl)
print("Track Images:", is_track)

#prompt user for the file directory. Will open as a popup window named "tk"
tk_root = tkinter.Tk()
print("Please open the directory that contains your Cyt5 tif files")
path_input = askdirectory(title='Select Input Folder with Cyt5 tif files')  # shows dialog box and return the path
print("Inputed path:", path_input)
tk_root.destroy()
path_output = path_input + '/Sorted'

if os.path.exists(path_output):
    path_out_new = input('Enter new output folder name')
    path_output = path_input + '/' + path_out_new
    os.makedirs(path_output)
#create a new  output directory
else:
    os.makedirs(path_output)

Current config:
Image Type: ph1
Fluorescent Images: False
Track Images: False
Please open the directory that contains your Cyt5 tif files
Inputed path: E:/Microscope Images/Cytation5/230419_191710_20230419_NHDFneo_P3_H2O2_Ascorbate/Plate 1


**Section 2: Execute functions to prepare folder structure for woundcompute**

In [38]:
# Create yaml file for image type
create_yaml(path_output, image_type)
print("Created .yaml file")


print("Sorting images into their corresponding stage position folders...")
#  Sort images in each basename folder into their corresponding stage position folders
sort_stage_pos_folders(path_input, path_output, image_type)
print("Completed")


Created .yaml file
Sorting images into their corresponding stage position folders...
Completed


**Section 3: Sort positions into parallelization folders**